In [2]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Dense, Input
import numpy as np
import pandas as pd

batch_size = 64
epochs = 100
latent_dim = 256
num_samples = 2700
filename = 'cl.csv'



In [3]:
# vectorize the data
input_texts = []
target_texts = []
input_characters = []
target_characters = []

df = pd.read_csv(filename)
input_texts = df['English'].values.tolist()
target_texts = df['Hindi'].values.tolist()

for i in range(len(target_texts)):
    target_texts[i] = '\t' + target_texts[i] + '\n'
    for char in input_texts[i]:
        if char not in input_characters:
            input_characters.append(char)
    for char in target_texts[i]:
        if char not in target_characters:
            target_characters.append(char)        

In [4]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_len = max([len(txt) for txt in input_texts])
max_decoder_seq_len = max([len(txt) for txt in target_texts])

In [5]:
print('i/p characters : {}'.format(len(input_characters)))
print('targert characters : {}'.format(len(target_characters)))
print('encoder characters : {}'.format(num_encoder_tokens))
print('decoder characters : {}'.format(num_decoder_tokens))
print('max encoder : {}'.format(max_encoder_seq_len))
print('max decoder : {}'.format(max_decoder_seq_len))

i/p characters : 70
targert characters : 87
encoder characters : 70
decoder characters : 87
max encoder : 107
max decoder : 123


In [6]:
input_token_index = dict([(char,i) for i,char in enumerate(input_characters)])
target_token_index = dict([(char,i) for i,char in enumerate(target_characters)])
print(input_token_index)
print(target_token_index)

In [7]:
encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_len, num_encoder_tokens), dtype='float32')
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_len, num_decoder_tokens), dtype='float32')
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_len, num_decoder_tokens), dtype='float32')

In [11]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t+1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        decoder_input_data[i,t,target_token_index[char]] = 1.
        if t > 0:
            decoder_target_data[i,t-1,target_token_index[char]] = 1.
    decoder_input_data[i,t+1:,target_token_index[' ']] = 1.
    decoder_target_data[i,t:,target_token_index[' ']] = 1.

In [13]:
encoder_input_data[0].shape

(107, 70)

In [15]:
# Define an input sequence and process it
encoder_inputs = Input(shape=(None,num_encoder_tokens))
encoder=LSTM(latent_dim,return_state=True)
encoder_outputs, state_h, state_c=encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states
encoder_states=[state_h,state_c]


In [16]:
# Set up decoder , using ` encoder_states` as initial state
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well . we don't use the 
# return states in the training model , but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense= Dense(num_decoder_tokens, activation='softmax')
decoder_outputs= decoder_dense(decoder_outputs)

In [ ]:
# Define the model that will turn 
# `encoder_input_data` and `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss = 'categorical_crossentropy', metrics= ['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size= batch_size, epochs = epochs , 
           validation_split=0.2)

Epoch 1/100
35/35 [==============================] - 47s 1s/step - loss: 1.8093 - accuracy: 0.7968 - val_loss: 1.5313 - val_accuracy: 0.6872
Epoch 2/100
35/35 [==============================] - 39s 1s/step - loss: 0.9375 - accuracy: 0.8088 - val_loss: 1.5407 - val_accuracy: 0.6897
Epoch 3/100
35/35 [==============================] - 40s 1s/step - loss: 0.8495 - accuracy: 0.8077 - val_loss: 1.5999 - val_accuracy: 0.6912
Epoch 4/100
35/35 [==============================] - 46s 1s/step - loss: 0.8017 - accuracy: 0.8093 - val_loss: 1.4324 - val_accuracy: 0.6969
Epoch 5/100
35/35 [==============================] - 54s 2s/step - loss: 0.7757 - accuracy: 0.8135 - val_loss: 1.2180 - val_accuracy: 0.7082
Epoch 6/100
35/35 [==============================] - 57s 2s/step - loss: 0.7073 - accuracy: 0.8242 - val_loss: 1.1463 - val_accuracy: 0.7087
Epoch 7/100
35/35 [==============================] - 55s 2s/step - loss: 0.6487 - accuracy: 0.8374 - val_loss: 1.0563 - val_accuracy: 0.7320
Epoch 8/100
3